In [212]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w7p1/mnist_test.csv
/kaggle/input/2022-ai-w7p1/mnist_train.csv
/kaggle/input/2022-ai-w7p1/submission.csv


In [213]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [214]:
train = pd.read_csv('../input/2022-ai-w7p1/mnist_train.csv')
test = pd.read_csv('../input/2022-ai-w7p1/mnist_test.csv')
submit = pd.read_csv('../input/2022-ai-w7p1/submission.csv')

In [215]:
trainX = train.drop(['Unnamed: 0', '784'], axis=1)
trainY = train['784']
testX = test.drop(['Unnamed: 0'], axis=1)

In [216]:
x_train = torch.FloatTensor((trainX / 255).to_numpy()).cuda()
y_train = torch.LongTensor(trainY.to_numpy()).cuda()
x_test = torch.FloatTensor((testX / 255).to_numpy()).cuda()

In [217]:
# data loader 설정
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = 100)

In [218]:
n = x_train.shape[1]
m = 512
linear1 = torch.nn.Linear(n, m, bias=True)
linear2 = torch.nn.Linear(m, m, bias=True)
linear3 = torch.nn.Linear(m, m, bias=True)
linear4 = torch.nn.Linear(m, m, bias=True)
linear5 = torch.nn.Linear(m, 10, bias=True)

relu = torch.nn.ReLU()

dropout = torch.nn.Dropout(p = 0.3)

In [219]:
# 자비에로 W 초기화 
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-0.0416,  0.0672, -0.0905,  ..., -0.0244, -0.0023, -0.0200],
        [ 0.1122, -0.0471,  0.0352,  ..., -0.0037, -0.0207,  0.1297],
        [ 0.0533, -0.0022,  0.1399,  ..., -0.0233, -0.0411, -0.0380],
        ...,
        [-0.0065, -0.1301,  0.0373,  ..., -0.1027, -0.0710, -0.0573],
        [-0.0422,  0.0770,  0.0814,  ...,  0.0139,  0.1599,  0.0426],
        [-0.0238,  0.0650,  0.0703,  ..., -0.0106,  0.0147, -0.0172]],
       requires_grad=True)

In [220]:
# 모델 생성 
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).cuda()

In [221]:
loss = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [222]:
# 모델 학습

toal_batch = len(train_loader)
model.train()

np_epochs = 25
for epoch in range(np_epochs) :
    for x_train_batch, y_train_batch in train_loader :
        # gradient 초기화
        optimizer.zero_grad()
        # forward 계산
        hypothesis = model(x_train_batch)
        # error 계산
        cost = loss(hypothesis, y_train_batch)
        # backward 계산
        cost.backward()
        # 가중치 계산
        optimizer.step()
        
    print(epoch, cost.item())
    
print("학습 종료")

0 0.2875349223613739
1 0.29029959440231323
2 0.23280243575572968
3 0.082787424325943
4 0.13734887540340424
5 0.11183425784111023
6 0.13603147864341736
7 0.09547457098960876
8 0.1440035104751587
9 0.07634325325489044
10 0.09640862792730331
11 0.08753552287817001
12 0.18115642666816711
13 0.0694180279970169
14 0.06891335546970367
15 0.06987286359071732
16 0.0561869852244854
17 0.03462157025933266
18 0.07031943649053574
19 0.04153839498758316
20 0.08373120427131653
21 0.061469774693250656
22 0.029986925423145294
23 0.07047860324382782
24 0.16386812925338745
학습 종료


In [223]:
with torch.no_grad() :
    model.eval()
    
    hypothesis = model(x_test)
    predict = torch.argmax(hypothesis, 1)

In [224]:
submit['Label'] = torch.Tensor.cpu(predict)

In [225]:
submit

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5


In [227]:
submit.to_csv('submit9.csv', index=False)